In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
url = 'https://raw.githubusercontent.com/isnikitin1/SalaryResearch/master/all_vacancies_clean.xlsx'
df = pd.read_excel(url)

### Этап 4: Построение модели. Оценка качества модели.

In [3]:
data = df.iloc[:, 2:].copy()

In [4]:
data.drop('alternate_url', axis=1, inplace=True)
data_cat = data.copy()

In [5]:
data = pd.get_dummies(data, drop_first=True)

In [6]:
X = data.drop('salary', axis=1)
y = data['salary']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

In [9]:
lin = LinearRegression()
lin.fit(X_train_s, y_train)
y_pred_train_lin = lin.predict(X_train_s)
y_pred_test_lin = lin.predict(X_test_s)

In [10]:
print('LinearRegression')
print(f'R2 train  {r2_score(y_train, y_pred_train_lin)} R2 test {r2_score(y_test, y_pred_test_lin)}')
print(f'MAE train {mean_absolute_error(y_train, y_pred_train_lin):.15f} MAE test {mean_absolute_error(y_test, y_pred_test_lin):.15f}')
print(f'MSE train {mean_squared_error(y_train, y_pred_train_lin):.15f} MSE test {mean_squared_error(y_test, y_pred_test_lin):.15f}')

LinearRegression
R2 train  1.0 R2 test 0.8354610294651986
MAE train 0.000000000124253 MAE test 12999.409068479344569
MSE train 0.000000000000000 MSE test 289732461.202279388904572


Качество модели оценим метриками:  
R2-коэффициент детерминации. Показывает, насколько хорошо модель объясняет дисперсию целевой переменной.  
MAE-средняя абсолютная ошибка. Измеряет среднюю величину абсолютных отклонений между предсказанными и фактическими значениями целевой переменной.  
MSE-средняя квадратичная ошибка. Измеряет среднее значение квадратов разностей между предсказанными и фактическими значениями.  

Модель LinearRegression сильно переобучена. Она идеально подходит к обучающим данным, но плохо обобщает новые данные. R2 на test наборе данных подтверждает, что модель плохо обобщает данные. Модель работает значительно хуже на новых данных, чем на тех, на которых она была обучена. MAE train близкое к нулю практически идеально предсказывает значения в обучающей выборке. Средняя абсолютная ошибка на test очень большая, что подтверждает переобучение. MSE train модель идеально предсказывает значения в обучающей выборке. Средняя квадратичная ошибка равна нулю. MSE test показывает, что в тестовом наборе данных есть значительные ошибки в предсказании.


In [11]:
lasso = LassoCV(cv=5, random_state=42)  
lasso.fit(X_train_s, y_train)
best_alpha = lasso.alpha_
print(f'Лучший alpha: {best_alpha}')

y_pred_train_las = lasso.predict(X_train_s)
y_pred_test_las = lasso.predict(X_test_s)

Лучший alpha: 39.70335816600147


In [12]:
print('LassoCV')
print(f'R2   train {r2_score(y_train, y_pred_train_las):.2f} R2 test {r2_score(y_test, y_pred_test_las):.2f}')
print(f'MAE  train {mean_absolute_error(y_train, y_pred_train_las):.2f} MAE test {mean_absolute_error(y_test, y_pred_test_las):.2f}')
print(f'MSE  train {mean_squared_error(y_train, y_pred_train_las):.2f} MSE test {mean_squared_error(y_test, y_pred_test_las):.2f}')

LassoCV
R2   train 1.00 R2 test 1.00
MAE  train 32.22 MAE test 34.16
MSE  train 1655.31 MSE test 1733.82


In [13]:
coef_la = lasso.coef_
coef_zero = np.sum(np.isclose(coef_la, 0))
print(f' Количество нулевых коэффициентов: {coef_zero}')

 Количество нулевых коэффициентов: 2141


Для улучшения качества предсказания и уменьшения переобучения используем модель LassoCV. С подбором оптимального параметра alpha который штрафует признаки с большими коэффициентами. За счет этого LassoCV отбирает наиболее важные признаки и упрощает модель, что помогает бороться с переобучением. Подбор параметра alpha делаем с помощью кросс-валидации.  
Модель занулила большинство признаков. Lasso выбрала только 2 наиболее важных признака для предсказания целевой переменной. Это означает, что эти два признака обладают наибольшей предсказательной силой. Несмотря на то, что модель показала отличный результат по всем используемым метрика, такое упрощение модели тоже не очень хорошо. Потому что модель использует минимальное количество признаков.


In [14]:
categorical_features = data_cat.select_dtypes(include=['object', 'bool']).columns.tolist()

X_cat = data_cat.drop('salary', axis=1)
y_cat = data_cat['salary']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, random_state=42)

In [15]:
param_grid = {'iterations': [1500],
              'learning_rate': [0.01, 0.1],
              'depth': [4],
              'l2_leaf_reg': [1, 3, 5],
              'bagging_temperature': [0.5, 1],
              'subsample': [0.6, 0.8]}

model = CatBoostRegressor(verbose=0, random_state=42, cat_features=categorical_features)  
kf = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_cat, y_train_cat, eval_set=[(X_test_cat, y_test_cat)], early_stopping_rounds=20) 
best_params = grid_search.best_params_
print('Лучшие параметры:', best_params)

Лучшие параметры: {'bagging_temperature': 0.5, 'depth': 4, 'iterations': 1500, 'l2_leaf_reg': 1, 'learning_rate': 0.01, 'subsample': 0.8}


In [16]:
final_model = CatBoostRegressor(**best_params, verbose=0, random_state=42, cat_features=categorical_features)
final_model.fit(X_train_cat, y_train_cat, eval_set=(X_test_cat, y_test_cat))

y_pred_train_grid = final_model.predict(X_train_cat)
y_pred_test_grid = final_model.predict(X_test_cat)

In [17]:
print('CatBoostRegressor с лучшими параметрами:')
print(f'R2  train {r2_score(y_train_cat, y_pred_train_grid):.2f} R2 test {r2_score(y_test_cat, y_pred_test_grid):.2f}')
print(f'MAE train {mean_absolute_error(y_train_cat, y_pred_train_grid):.2f} MAE test {mean_absolute_error(y_test_cat, y_pred_test_grid):.2f}')
print(f'MSE train {mean_squared_error(y_train_cat, y_pred_train_grid):.2f} MSE test {mean_squared_error(y_test_cat, y_pred_test_grid):.2f}')

CatBoostRegressor с лучшими параметрами:
R2  train 1.00 R2 test 0.99
MAE train 417.93 MAE test 836.36
MSE train 372045.82 MSE test 18396755.74


Для улучшения качества предсказания и снижения риска переобучения используем модель CatBoostRegressor. Преимуществом данной модели является отсутствие необходимости предварительного кодирования номинативных признаков и масштабирования данных, что упрощает процесс подготовки данных. С помощью поиска оптимальных параметров по сетке (GridSearchCV) подбираем гиперпараметры.  
Модель CatBoostRegressor демонстрирует высокие показатели качества на тренировочном и тестовом наборах данных. Так как мы предсказываем заработную плату аналитика и среднее абсолютное отклонение предсказанных значений заработной платы от фактических эквивалентно стоимости пары чашек кофе. Я считаю, что модель CatBoostRegressor подходит для решения данной задачи.
